In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import time
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from models import SRResNet
from datasets import SRDataset
from utils import *
import pandas as pd
import numpy as np
import csv

In [3]:
from datetime import datetime
start_time = datetime.now()

# Data parameters
data_folder = './'  # folder with JSON data files
crop_size = 96  # crop size of target HR images
scaling_factor = 2  # the scaling factor for the generator; the input LR images will be downsampled from the target HR images by this factor

# Model parameters
large_kernel_size = 9  # kernel size of the first and last convolutions which transform the inputs and outputs
small_kernel_size = 3  # kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
n_channels = 64  # number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
n_blocks = 16  # number of residual blocks

# Learning parameters
checkpoint = None  # path to model checkpoint, None if none
batch_size = 32  # batch size
start_epoch = 0  # start at this epoch
iterations = 1e6  # number of training iterations
workers = 0  # number of workers for loading data in the DataLoader
print_freq = 1  # print training status once every __ batches
lr = 1e-4  # learning rate
grad_clip = None  # clip if gradients are exploding
index = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cudnn.benchmark = True


def main():
    """
    Training.
    """
    global start_epoch, epoch, checkpoint

    # Initialize model or load checkpoint
    if checkpoint is None:
        model = SRResNet(large_kernel_size=large_kernel_size, small_kernel_size=small_kernel_size,
                         n_channels=n_channels, n_blocks=n_blocks, scaling_factor=scaling_factor)
        # Initialize the optimizer
        optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                     lr=lr)

    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']

    # Move to default device
    model = model.to(device)
    criterion = nn.MSELoss().to(device)

    # Custom dataloaders
    train_dataset = SRDataset(data_folder,
                              split='train',
                              crop_size=crop_size,
                              scaling_factor=scaling_factor,
                              lr_img_type='imagenet-norm',
                              hr_img_type='[-1, 1]')
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers,
                                               pin_memory=True)  # note that we're passing the collate function here

    # Total number of epochs to train for
    epochs = int(iterations // len(train_loader) + 1)

    
    csvfile = open('Model SRResNet Label Upscale X2/Loss SRResNet Label Epoch 100 Batch Size 32 Upscale X2_srresnet.csv','w')
    filewriter = csv.writer(csvfile, delimiter = ',', quotechar = '|',quoting = csv.QUOTE_MINIMAL)
    filewriter.writerow(['Nomor','Epoch','Loss'])

    # Epochs
    for epoch in range(start_epoch, 100):
        # One epoch's training
        train(train_loader=train_loader,
              model=model,
              criterion=criterion,
              optimizer=optimizer,
              epoch=epoch,
              idx = index,
              filewriter = filewriter)

        # Save checkpoint
        torch.save({'epoch': epoch,
                    'model': model,
                    'optimizer': optimizer},
                   'Model SRResNet Label Upscale X2/Model SRResNet Label Epoch 100 Batch Size 32 Upscale x2_srresnet.pth.tar')

from datetime import datetime
start_time = datetime.now()
# do your work here
result_loss = pd.DataFrame()


def train(train_loader, model, criterion, optimizer, epoch, idx, filewriter):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: content loss function (Mean Squared-Error loss)
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    model.train()  # training mode enables batch normalization

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
            
    # Batches
    for i, (lr_imgs, hr_imgs) in enumerate(train_loader):
        data_time.update(time.time() - start)

        # Move to default device
        lr_imgs = lr_imgs.to(device)  # (batch_size (N), 3, 24, 24), imagenet-normed
        hr_imgs = hr_imgs.to(device)  # (batch_size (N), 3, 96, 96), in [-1, 1]

        # Forward prop.
        sr_imgs = model(lr_imgs)  # (N, 3, 96, 96), in [-1, 1]

        # Loss
        loss = criterion(sr_imgs, hr_imgs)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Clip gradients, if necessary
        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        # Update model
        optimizer.step()

        # Keep track of loss
        losses.update(loss.item(), lr_imgs.size(0))

        # Keep track of batch time
        batch_time.update(time.time() - start)

        # Reset start time
        start = time.time()

        # Print status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]----'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})----'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})----'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(epoch, i, len(train_loader),
                                                                    batch_time=batch_time,
                                                                    data_time=data_time, loss=losses))
            idx = idx + 1
            filewriter.writerow([idx,epoch,'{loss.val:.4f}'.format(loss = losses)])
            
            
            
        del lr_imgs, hr_imgs, sr_imgs  # free some memory since their histories may be stored


if __name__ == '__main__':
    main()

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch: [0][0/72]----Batch Time 2.758 (2.758)----Data Time 0.422 (0.422)----Loss 0.4370 (0.4370)
Epoch: [0][1/72]----Batch Time 0.309 (1.533)----Data Time 0.174 (0.298)----Loss 0.2948 (0.3659)
Epoch: [0][2/72]----Batch Time 0.235 (1.101)----Data Time 0.124 (0.240)----Loss 0.1947 (0.3088)
Epoch: [0][3/72]----Batch Time 0.192 (0.874)----Data Time 0.095 (0.203)----Loss 0.1200 (0.2616)
Epoch: [0][4/72]----Batch Time 0.219 (0.743)----Data Time 0.119 (0.187)----Loss 0.0702 (0.2233)
Epoch: [0][5/72]----Batch Time 0.174 (0.648)----Data Time 0.077 (0.168)----Loss 0.0596 (0.1960)
Epoch: [0][6/72]----Batch Time 0.179 (0.581)----Data Time 0.084 (0.156)----Loss 0.0482 (0.1749)
Epoch: [0][7/72]----Batch Time 0.169 (0.529)----Data Time 0.075 (0.146)----Loss 0.0477 (0.1590)
Epoch: [0][8/72]----Batch Time 0.191 (0.492)----Data Time 0.095 (0.140)----Loss 0.0418 (0.1460)
Epoch: [0][9/72]----Batch Time 0.170 (0.460)----Data Time 0.073 (0.134)----Loss 0.0423 (0.1356)
Epoch: [0][10/72]----Batch Time 0.183 (0

Epoch: [1][14/72]----Batch Time 0.146 (0.149)----Data Time 0.040 (0.047)----Loss 0.0098 (0.0100)
Epoch: [1][15/72]----Batch Time 0.159 (0.149)----Data Time 0.059 (0.048)----Loss 0.0117 (0.0101)
Epoch: [1][16/72]----Batch Time 0.133 (0.148)----Data Time 0.042 (0.047)----Loss 0.0086 (0.0101)
Epoch: [1][17/72]----Batch Time 0.142 (0.148)----Data Time 0.049 (0.047)----Loss 0.0092 (0.0100)
Epoch: [1][18/72]----Batch Time 0.141 (0.148)----Data Time 0.044 (0.047)----Loss 0.0084 (0.0099)
Epoch: [1][19/72]----Batch Time 0.139 (0.147)----Data Time 0.048 (0.047)----Loss 0.0087 (0.0099)
Epoch: [1][20/72]----Batch Time 0.141 (0.147)----Data Time 0.049 (0.047)----Loss 0.0109 (0.0099)
Epoch: [1][21/72]----Batch Time 0.135 (0.146)----Data Time 0.042 (0.047)----Loss 0.0094 (0.0099)
Epoch: [1][22/72]----Batch Time 0.138 (0.146)----Data Time 0.045 (0.047)----Loss 0.0094 (0.0099)
Epoch: [1][23/72]----Batch Time 0.140 (0.146)----Data Time 0.048 (0.047)----Loss 0.0110 (0.0099)
Epoch: [1][24/72]----Batch Tim

Epoch: [2][28/72]----Batch Time 0.148 (0.149)----Data Time 0.048 (0.049)----Loss 0.0062 (0.0066)
Epoch: [2][29/72]----Batch Time 0.166 (0.149)----Data Time 0.052 (0.049)----Loss 0.0058 (0.0066)
Epoch: [2][30/72]----Batch Time 0.141 (0.149)----Data Time 0.042 (0.049)----Loss 0.0054 (0.0066)
Epoch: [2][31/72]----Batch Time 0.132 (0.149)----Data Time 0.039 (0.048)----Loss 0.0061 (0.0066)
Epoch: [2][32/72]----Batch Time 0.147 (0.149)----Data Time 0.048 (0.048)----Loss 0.0064 (0.0066)
Epoch: [2][33/72]----Batch Time 0.142 (0.148)----Data Time 0.044 (0.048)----Loss 0.0058 (0.0065)
Epoch: [2][34/72]----Batch Time 0.157 (0.149)----Data Time 0.058 (0.048)----Loss 0.0058 (0.0065)
Epoch: [2][35/72]----Batch Time 0.140 (0.148)----Data Time 0.044 (0.048)----Loss 0.0070 (0.0065)
Epoch: [2][36/72]----Batch Time 0.154 (0.148)----Data Time 0.053 (0.048)----Loss 0.0062 (0.0065)
Epoch: [2][37/72]----Batch Time 0.148 (0.148)----Data Time 0.053 (0.049)----Loss 0.0076 (0.0065)
Epoch: [2][38/72]----Batch Tim

Epoch: [3][42/72]----Batch Time 0.147 (0.146)----Data Time 0.055 (0.049)----Loss 0.0045 (0.0051)
Epoch: [3][43/72]----Batch Time 0.141 (0.146)----Data Time 0.048 (0.049)----Loss 0.0045 (0.0051)
Epoch: [3][44/72]----Batch Time 0.140 (0.146)----Data Time 0.042 (0.048)----Loss 0.0047 (0.0051)
Epoch: [3][45/72]----Batch Time 0.136 (0.145)----Data Time 0.045 (0.048)----Loss 0.0038 (0.0051)
Epoch: [3][46/72]----Batch Time 0.140 (0.145)----Data Time 0.044 (0.048)----Loss 0.0051 (0.0051)
Epoch: [3][47/72]----Batch Time 0.138 (0.145)----Data Time 0.043 (0.048)----Loss 0.0056 (0.0051)
Epoch: [3][48/72]----Batch Time 0.145 (0.145)----Data Time 0.051 (0.048)----Loss 0.0035 (0.0050)
Epoch: [3][49/72]----Batch Time 0.152 (0.145)----Data Time 0.044 (0.048)----Loss 0.0055 (0.0051)
Epoch: [3][50/72]----Batch Time 0.153 (0.145)----Data Time 0.053 (0.048)----Loss 0.0045 (0.0050)
Epoch: [3][51/72]----Batch Time 0.145 (0.145)----Data Time 0.050 (0.048)----Loss 0.0046 (0.0050)
Epoch: [3][52/72]----Batch Tim

Epoch: [4][56/72]----Batch Time 0.162 (0.147)----Data Time 0.049 (0.048)----Loss 0.0045 (0.0047)
Epoch: [4][57/72]----Batch Time 0.129 (0.146)----Data Time 0.038 (0.048)----Loss 0.0034 (0.0047)
Epoch: [4][58/72]----Batch Time 0.140 (0.146)----Data Time 0.044 (0.048)----Loss 0.0041 (0.0046)
Epoch: [4][59/72]----Batch Time 0.140 (0.146)----Data Time 0.044 (0.048)----Loss 0.0049 (0.0047)
Epoch: [4][60/72]----Batch Time 0.144 (0.146)----Data Time 0.041 (0.048)----Loss 0.0041 (0.0046)
Epoch: [4][61/72]----Batch Time 0.135 (0.146)----Data Time 0.043 (0.048)----Loss 0.0053 (0.0047)
Epoch: [4][62/72]----Batch Time 0.139 (0.146)----Data Time 0.042 (0.048)----Loss 0.0034 (0.0046)
Epoch: [4][63/72]----Batch Time 0.152 (0.146)----Data Time 0.055 (0.048)----Loss 0.0069 (0.0047)
Epoch: [4][64/72]----Batch Time 0.147 (0.146)----Data Time 0.044 (0.048)----Loss 0.0048 (0.0047)
Epoch: [4][65/72]----Batch Time 0.147 (0.146)----Data Time 0.056 (0.048)----Loss 0.0044 (0.0047)
Epoch: [4][66/72]----Batch Tim

...

Epoch: [94][58/72]----Batch Time 0.148 (0.148)----Data Time 0.054 (0.050)----Loss 0.0011 (0.0012)
Epoch: [94][59/72]----Batch Time 0.148 (0.148)----Data Time 0.043 (0.050)----Loss 0.0010 (0.0012)
Epoch: [94][60/72]----Batch Time 0.152 (0.148)----Data Time 0.051 (0.050)----Loss 0.0020 (0.0012)
Epoch: [94][61/72]----Batch Time 0.138 (0.148)----Data Time 0.045 (0.050)----Loss 0.0011 (0.0012)
Epoch: [94][62/72]----Batch Time 0.137 (0.148)----Data Time 0.043 (0.050)----Loss 0.0008 (0.0012)
Epoch: [94][63/72]----Batch Time 0.144 (0.148)----Data Time 0.041 (0.049)----Loss 0.0011 (0.0012)
Epoch: [94][64/72]----Batch Time 0.131 (0.148)----Data Time 0.038 (0.049)----Loss 0.0011 (0.0012)
Epoch: [94][65/72]----Batch Time 0.143 (0.147)----Data Time 0.049 (0.049)----Loss 0.0011 (0.0012)
Epoch: [94][66/72]----Batch Time 0.161 (0.148)----Data Time 0.050 (0.049)----Loss 0.0011 (0.0012)
Epoch: [94][67/72]----Batch Time 0.154 (0.148)----Data Time 0.060 (0.049)----Loss 0.0010 (0.0012)
Epoch: [94][68/72]--

Epoch: [95][70/72]----Batch Time 0.132 (0.145)----Data Time 0.040 (0.048)----Loss 0.0012 (0.0011)
Epoch: [95][71/72]----Batch Time 0.083 (0.144)----Data Time 0.023 (0.047)----Loss 0.0013 (0.0012)
Epoch: [96][0/72]----Batch Time 0.149 (0.149)----Data Time 0.037 (0.037)----Loss 0.0011 (0.0011)
Epoch: [96][1/72]----Batch Time 0.175 (0.162)----Data Time 0.079 (0.058)----Loss 0.0016 (0.0013)
Epoch: [96][2/72]----Batch Time 0.133 (0.152)----Data Time 0.041 (0.052)----Loss 0.0015 (0.0014)
Epoch: [96][3/72]----Batch Time 0.156 (0.153)----Data Time 0.050 (0.052)----Loss 0.0012 (0.0013)
Epoch: [96][4/72]----Batch Time 0.150 (0.152)----Data Time 0.049 (0.051)----Loss 0.0011 (0.0013)
Epoch: [96][5/72]----Batch Time 0.164 (0.154)----Data Time 0.054 (0.052)----Loss 0.0011 (0.0013)
Epoch: [96][6/72]----Batch Time 0.187 (0.159)----Data Time 0.046 (0.051)----Loss 0.0014 (0.0013)
Epoch: [96][7/72]----Batch Time 0.136 (0.156)----Data Time 0.043 (0.050)----Loss 0.0024 (0.0014)
Epoch: [96][8/72]----Batch T

Epoch: [97][10/72]----Batch Time 0.145 (0.143)----Data Time 0.044 (0.044)----Loss 0.0013 (0.0012)
Epoch: [97][11/72]----Batch Time 0.134 (0.142)----Data Time 0.040 (0.044)----Loss 0.0018 (0.0012)
Epoch: [97][12/72]----Batch Time 0.138 (0.142)----Data Time 0.040 (0.044)----Loss 0.0009 (0.0012)
Epoch: [97][13/72]----Batch Time 0.153 (0.143)----Data Time 0.047 (0.044)----Loss 0.0012 (0.0012)
Epoch: [97][14/72]----Batch Time 0.158 (0.144)----Data Time 0.053 (0.044)----Loss 0.0014 (0.0012)
Epoch: [97][15/72]----Batch Time 0.150 (0.144)----Data Time 0.050 (0.045)----Loss 0.0010 (0.0012)
Epoch: [97][16/72]----Batch Time 0.142 (0.144)----Data Time 0.049 (0.045)----Loss 0.0010 (0.0012)
Epoch: [97][17/72]----Batch Time 0.132 (0.143)----Data Time 0.038 (0.045)----Loss 0.0008 (0.0012)
Epoch: [97][18/72]----Batch Time 0.153 (0.144)----Data Time 0.042 (0.045)----Loss 0.0017 (0.0012)
Epoch: [97][19/72]----Batch Time 0.150 (0.144)----Data Time 0.046 (0.045)----Loss 0.0009 (0.0012)
Epoch: [97][20/72]--

Epoch: [98][22/72]----Batch Time 0.140 (0.149)----Data Time 0.039 (0.048)----Loss 0.0010 (0.0013)
Epoch: [98][23/72]----Batch Time 0.176 (0.150)----Data Time 0.045 (0.048)----Loss 0.0010 (0.0012)
Epoch: [98][24/72]----Batch Time 0.165 (0.150)----Data Time 0.067 (0.049)----Loss 0.0007 (0.0012)
Epoch: [98][25/72]----Batch Time 0.143 (0.150)----Data Time 0.039 (0.048)----Loss 0.0022 (0.0013)
Epoch: [98][26/72]----Batch Time 0.136 (0.149)----Data Time 0.041 (0.048)----Loss 0.0011 (0.0013)
Epoch: [98][27/72]----Batch Time 0.142 (0.149)----Data Time 0.050 (0.048)----Loss 0.0008 (0.0012)
Epoch: [98][28/72]----Batch Time 0.131 (0.149)----Data Time 0.038 (0.048)----Loss 0.0010 (0.0012)
Epoch: [98][29/72]----Batch Time 0.155 (0.149)----Data Time 0.053 (0.048)----Loss 0.0010 (0.0012)
Epoch: [98][30/72]----Batch Time 0.166 (0.149)----Data Time 0.051 (0.048)----Loss 0.0010 (0.0012)
Epoch: [98][31/72]----Batch Time 0.157 (0.150)----Data Time 0.064 (0.048)----Loss 0.0011 (0.0012)
Epoch: [98][32/72]--

Epoch: [99][34/72]----Batch Time 0.142 (0.148)----Data Time 0.044 (0.048)----Loss 0.0013 (0.0012)
Epoch: [99][35/72]----Batch Time 0.136 (0.148)----Data Time 0.038 (0.048)----Loss 0.0012 (0.0012)
Epoch: [99][36/72]----Batch Time 0.142 (0.148)----Data Time 0.048 (0.048)----Loss 0.0009 (0.0011)
Epoch: [99][37/72]----Batch Time 0.162 (0.148)----Data Time 0.046 (0.048)----Loss 0.0008 (0.0011)
Epoch: [99][38/72]----Batch Time 0.144 (0.148)----Data Time 0.047 (0.048)----Loss 0.0009 (0.0011)
Epoch: [99][39/72]----Batch Time 0.143 (0.148)----Data Time 0.049 (0.048)----Loss 0.0010 (0.0011)
Epoch: [99][40/72]----Batch Time 0.151 (0.148)----Data Time 0.057 (0.048)----Loss 0.0009 (0.0011)
Epoch: [99][41/72]----Batch Time 0.221 (0.150)----Data Time 0.118 (0.049)----Loss 0.0014 (0.0011)
Epoch: [99][42/72]----Batch Time 0.136 (0.150)----Data Time 0.038 (0.049)----Loss 0.0010 (0.0011)
Epoch: [99][43/72]----Batch Time 0.137 (0.149)----Data Time 0.043 (0.049)----Loss 0.0008 (0.0011)
Epoch: [99][44/72]--

In [4]:
# from utils import create_data_lists

# if __name__ == '__main__':
#     create_data_lists(train_folders=['Kumpulan Citra per Rekam Medis/Json Vid Unseen/train/'],
#                       test_folders=['Kumpulan Citra per Rekam Medis/Json Vid Unseen/test/'],
#                       min_size=100,
#                       output_folder='./')
